# 1033 Data 
### Summary
 - Download raw set
 - Group set by State, Item Name, Ship Date, and NSN (National Stock Number)
 - Derive PSC (Product and Suppy Codes) to categorize items.
 - Download PSC Manual to relate descriptions to PSC codes

In [15]:
import pandas as pd
program_df = pd.read_csv('https://query.data.world/s/jblkhnme7uibgq4golet263yn5jcvj')
#program_df.head()

In [16]:
# Create a new DF grouping by State, Item Name, Ship Date, NSN.
# Get sum of Quantity
summary_df = program_df.groupby(by = ['Year','State','State_Ab','County', 'Item Name', 'Ship Date','NSN'])['Quantity'].sum()
summary_df = pd.DataFrame(summary_df)
summary_df.reset_index(inplace= True)
#summary_df.head()

In [17]:
# Having a mental block...I'm sure there is a more elegant way to add the sum of total cost to the DF above.  But this works.
# Make a new DF and merge it into summary_df on the non-aggregated attributes.
merge_can = program_df.groupby(by= ['Year','State','State_Ab','County', 'Item Name', 'Ship Date','NSN'])['Total.Cost'].sum()
merge_can = pd.DataFrame(merge_can)
merge_can.head()
merge_can.reset_index(inplace = True)
summary_df = summary_df.merge(merge_can, how= "inner", on = ['Year','State','State_Ab','County', 'Item Name', 'Ship Date','NSN'])
# summary_df.head()

In [18]:
# Per "Federal Procurement Data System" manual, 
# the PSC code can be used to categorize the Item Names and are the first 4 char of the NSN
# Get the PSC code from the NSN
summary_df['psc'] = summary_df['NSN'].str.slice(start=0, stop=4, step=None)

In [19]:
summary_df["psc"]=pd.to_numeric(summary_df["psc"], errors='coerce')


In [20]:
summary_df.head()


,Year,State,State_Ab,County,Item Name,Ship Date,NSN,Quantity,Total.Cost,psc
0,2006,ALABAMA,AL,AUTAUGA,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,4,1996.00,1005
1,2006,ALABAMA,AL,BALDWIN,"PISTOL,CALIBER .45,AUTOMATIC",11/15/2006,1005-00-726-5655,24,1409.04,1005
2,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,2,998.00,1005
3,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",4/15/2006,1005-00-073-9421,4,1996.00,1005
4,2006,ALABAMA,AL,BALDWIN,"RIFLE,7.62 MILLIMETER",11/15/2006,1005-00-589-1271,10,1380.00,1005


## Enhance Data with PSC Manual Data

### Steps:
 - Load PSC Codes
 - Load Group Codes
 - Merge PSC and Group data
 - Merge that set into Summary data
 

In [21]:
psc_df = pd.read_csv("psc_codes.csv")
#psc_df.head()

In [22]:
group_df = pd.read_csv("group_codes.csv" )
#group_df.head()

In [23]:
# use an inner join.  How many records drop?
# psc to numeric on both psc_df and summary_df.  There is an issue merging otherwise.
psc_df = psc_df.merge(group_df, how= 'inner', on = "group")
psc_df["psc"]=pd.to_numeric(psc_df["psc"], errors='coerce')


In [24]:
summary_df = summary_df.merge(psc_df, how = 'inner', on = 'psc')

In [25]:
summary_df.head()

,Year,State,State_Ab,County,Item Name,Ship Date,NSN,Quantity,Total.Cost,psc,group,psc_desc,psc_category,group_desc,group_note
0,2006,ALABAMA,AL,AUTAUGA,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,4,1996.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...
1,2006,ALABAMA,AL,BALDWIN,"PISTOL,CALIBER .45,AUTOMATIC",11/15/2006,1005-00-726-5655,24,1409.04,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...
2,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,2,998.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...
3,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",4/15/2006,1005-00-073-9421,4,1996.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...
4,2006,ALABAMA,AL,BALDWIN,"RIFLE,7.62 MILLIMETER",11/15/2006,1005-00-589-1271,10,1380.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...


In [26]:
geo_df = pd.read_csv("geo.csv")
geo_df.head()

,Lat,Lon,State_Ab,County
0,40.922326,-72.637078,NY,SUFFOLK
1,18.165273,-66.722583,PR,ADJUNTAS
2,18.393103,-67.180953,PR,AGUADA
3,18.455913,-67.145780,PR,AGUADILLA
4,18.172947,-66.944111,PR,MARICAO


In [27]:
summary_df = summary_df.merge(geo_df, how= "left", on = ['State_Ab','County'])

In [28]:
summary_df.head()
summary_df.to_csv("summary.csv")

## Plot Findings

 - Finding 1
 - Finding 2

In [29]:
import pandas as pd
summary_df = pd.read_csv("summary.csv")
summary_df

,Unnamed: 0,Year,State,State_Ab,County,Item Name,Ship Date,NSN,Quantity,Total.Cost,psc,group,psc_desc,psc_category,group_desc,group_note,Lat,Lon
0,0,2006,ALABAMA,AL,AUTAUGA,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,4,1996.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,32.450328,-86.721712
1,1,2006,ALABAMA,AL,BALDWIN,"PISTOL,CALIBER .45,AUTOMATIC",11/15/2006,1005-00-726-5655,24,1409.04,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371
2,2,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,2,998.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371
3,3,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",4/15/2006,1005-00-073-9421,4,1996.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371
4,4,2006,ALABAMA,AL,BALDWIN,"RIFLE,7.62 MILLIMETER",11/15/2006,1005-00-589-1271,10,1380.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371
5,5,2006,ALABAMA,AL,BARBOUR,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,10,4990.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,31.805322,-85.421299
6,6,2006,ALABAMA,AL,BIBB,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,8,3992.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,32.989499,-87.220392
7,7,2006,ALABAMA,AL,BLOUNT,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,2,998.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,33.922162,-86.449262
8,8,2006,ALABAMA,AL,BLOUNT,"RIFLE,7.62 MILLIMETER",11/15/2006,1005-00-589-1271,1,138.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,33.922162,-86.449262
9,9,2006,ALABAMA,AL,BLOUNT,"RIFLE,7.62 MILLIMETER",5/15/2006,1005-00-589-1271,2,276.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,33.922162,-86.449262


In [30]:
import plotly.plotly as py
import plotly.graph_objs as go
from config import api_key, username
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#plotly.tools.set_credentials_file(username='mansplanish', api_key='BkaNgNAvB72omUPLQVOJ')
py.sign_in(username, api_key)
init_notebook_mode(connected=True)



In [31]:
# Plot Quantity of Guns, Ammunition, and other Weapons over time
# Limit dataframe to these categories

military_df =  summary_df[summary_df['group'].isin(['10', '13'])]

military_df.head()

,Unnamed: 0,Year,State,State_Ab,County,Item Name,Ship Date,NSN,Quantity,Total.Cost,psc,group,psc_desc,psc_category,group_desc,group_note,Lat,Lon
0,0,2006,ALABAMA,AL,AUTAUGA,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,4,1996.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,32.450328,-86.721712
1,1,2006,ALABAMA,AL,BALDWIN,"PISTOL,CALIBER .45,AUTOMATIC",11/15/2006,1005-00-726-5655,24,1409.04,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371
2,2,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",11/15/2006,1005-00-073-9421,2,998.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371
3,3,2006,ALABAMA,AL,BALDWIN,"RIFLE,5.56 MILLIMETER",4/15/2006,1005-00-073-9421,4,1996.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371
4,4,2006,ALABAMA,AL,BALDWIN,"RIFLE,7.62 MILLIMETER",11/15/2006,1005-00-589-1271,10,1380.00,1005,10,Guns,through 30mm,Weapons,This group includes combat weapons as well as ...,30.763492,-87.756371


In [38]:
# Set up variables for next 3 plots
st = military_df['State']
x = military_df['Year']
y = military_df['Total.Cost']
sy = y/5000
q = military_df['Quantity']
sq = q/5000

In [36]:
trace1 = go.Bar(
    x=x,
    y=y,
    #mode = 'markers',
    name='Total Cost'
)
trace2 = go.Bar(
    x=x,
    y=q,
   # mode = 'markers',
    name='Quantity',
    yaxis='y2'
)

data = [trace1, trace2]
layout = go.Layout(
    title='Total Cost and Quantity over Time',
    

    yaxis=dict(
        title='Total Cost'
    ),
    yaxis2=dict(
        title='Quantity',
        titlefont=dict(
            color='rgb(148, 103, 189)'
        ),
        tickfont=dict(
            color='rgb(148, 103, 189)'
        ),
        overlaying='y',
        side='right'
    )
)
fig = go.Figure(data=data, layout=layout)
#plot_url = py.plot(fig, filename='multiple-axes-double')
py.iplot(fig, filename='barchart')



In [62]:
military_df.sort_values('Total.Cost', ascending=False)
trace3 = go.Bar(
    x= military_df['State_Ab'],
    y= military_df['Total.Cost'],
    name='Total Cost',
   
)

data2 = [trace3]
layout2 = go.Layout(
    title='Total Cost and Quantity in US',
    yaxis=dict(title='Total Cost'))

fig = go.Figure(data=data2, layout=layout2)
#plot_url = py.plot(fig, filename='multiple-axes-double')
py.iplot(fig, filename='State Bar')



In [ ]:
# Add color, tweak the alpha
# Add lables


y = military_df["Quantity"]
q = military_df["Total.Cost"]/5000

trace0 = go.Scatter(
    x= x,
    y= y,
    mode='markers',
    marker=dict(
        size=q,
    )
)

data2 = [trace0]
#plot(data2)
py.iplot(data2, filename='bubblechart-size')


In [69]:
import plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=military_df['State_Ab'],
    y=y,
    name='Total Cost',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)
trace2 = go.Bar(
    x=military_df['State_Ab'],
    y=q,
    name='Quantity',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)
data = [trace1, trace2]
layout = go.Layout(
    title='US Export of Plastic Scrap',
    xaxis=dict(
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='USD (millions)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
            
        )
        ,
        overlaying='y',
        side='right'
    ),
    
    barmode='group',
   # bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='style-bar')


PlotlyRequestError: Hi there, you've reached the threshold of 100 combined image exports and chart saves per day. If you need to raise your daily limit, consider upgrading to a Student or Personal Plan
see: https://plot.ly/products/cloud